In [230]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [231]:
import pandas as pd
from numpy import nan
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation

from sklearn import svm
from sklearn import decomposition
from sklearn.neighbors import KNeighborsClassifier


In [232]:
# Set the global default size of matplotlib figures
#plt.rc('figure', figsize=(10,5))

In [233]:
train_data = pd.read_csv('./Data/ML Data Collection.csv')

In [234]:
scalar = preprocessing.MinMaxScaler(feature_range=(0,1))

#Mapping priority list into numbers
priority_list = sorted(train_data['Priority'].unique())

float_list = [float(i) for i in range(0,len(priority_list))]
priority_mapping = dict(zip(priority_list,float_list))
train_data['priority_val'] = train_data['Priority'].map(priority_mapping)

train_data['priority_val'] = (scalar.fit_transform(train_data['priority_val'].reshape(-1, 1))).ravel([1])

In [235]:
# APT normalization
train_data['APT'] = (scalar.fit_transform(train_data['APT'].reshape(-1, 1))).ravel([1])

In [236]:
# TSLR normalization
train_data['TSLR'] = (scalar.fit_transform(train_data['TSLR'].reshape(-1, 1))).ravel([1])

In [237]:
# Check if 'Calls from Customer' needs to be normalized
floatCallsArray =  np.asarray([float(i) for i in train_data['Calls from Customer']])
train_data['Calls from Customer'] = (scalar.fit_transform(floatCallsArray.reshape(-1, 1))).ravel([1])

In [238]:
# Check if 'Number of Info to SAP from last TSLR' needs to be normalized
floatNumInfoArray =  np.asarray([float(i) for i in train_data['Number of Info to SAP from last TSLR']])
train_data['Number of Info to SAP from last TSLR'] = (scalar.fit_transform(floatNumInfoArray.reshape(-1, 1))).ravel([1])

In [239]:
# Check if 'Frequent changes in processor' needs to be normalized - might not be, need to check the whole data
floatFreqChangeArray =  np.asarray([float(i) for i in train_data['Frequent changes in processor']])
train_data['Frequent changes in processor'] = (scalar.fit_transform(floatFreqChangeArray.reshape(-1, 1))).ravel([1])

In [240]:
# Check if 'Number of Processors' needs to be normalized - might not be, need to check the whole data
floatNumProcArray =  np.asarray([float(i) for i in train_data['Number of Processors']])
train_data['Number of Processors'] = (scalar.fit_transform(floatNumProcArray.reshape(-1, 1))).ravel([1])

In [241]:
#Mapping callback list into numbers
#callback_list = sorted(train_data['Customer call back request'].unique())
#callback_mapping = dict(zip(callback_list,range(0,len(callback_list))))
#train_data['callback_val'] = train_data['Customer call back request'].map(callback_mapping)

In [242]:
#Mapping core business list into numbers
core_business_list = sorted(train_data['Core Business affected?'].unique())
core_business_list.remove(nan)
core_business_mapping = dict(zip(core_business_list,range(0,len(core_business_list))))
train_data['core_business_val'] = train_data['Core Business affected?'].map(core_business_mapping)

In [243]:
train_data['core_business_val'] = train_data['core_business_val'].fillna(train_data['core_business_val'].median())


In [244]:
#Mapping Go live list into numbers
go_live_list = sorted(train_data['Go live Affected?'].unique())
go_live_list.remove(nan)
go_live_mapping = dict(zip(go_live_list,range(0,len(go_live_list))))
train_data['go_live_val'] = train_data['Go live Affected?'].map(go_live_mapping)

In [245]:
train_data['go_live_val'] = train_data['go_live_val'].fillna(train_data['go_live_val'].median())

In [246]:
#Either normalize or remove Num of users affected

In [247]:
#Mapping Work around list into numbers
work_around_list = sorted(train_data['Work around available?'].unique())
work_around_list.remove(nan)
work_around_mapping = dict(zip(work_around_list,range(0,len(work_around_list))))
train_data['work_around_val'] = train_data['Work around available?'].map(work_around_mapping)

In [248]:
train_data['work_around_val'] = train_data['work_around_val'].fillna(train_data['work_around_val'].median())

In [249]:
#Mapping escalation states into numbers
escStages = sorted(train_data['Stage of Escalation'].unique())
states_mapping = dict(zip(escStages,range(0,len(escStages))))
train_data['escStages_val'] = train_data['Stage of Escalation'].map(states_mapping)

In [250]:
train_data = train_data.drop(['Stage of Escalation', 'Priority', 'Customer call back request', 'Core Business affected?',
                             'Go live Affected?', 'Work around available?'], axis=1)

In [251]:
train_data = train_data.drop(['Num of users affected', 'Business Impact mentioned?'], axis=1)

In [252]:
#useful
#train_data

In [486]:
train_values = train_data.values
X = train_values[:,:-1]
Y = train_values[:,-1:]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,Y, test_size=0.33, random_state=0)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(229L, 10L) (113L, 10L) (229L, 1L) (113L, 1L)


In [487]:
#rf = RandomForestClassifier(n_estimators=800,max_depth=50)
rf = RandomForestClassifier(n_estimators=870, min_samples_leaf =8, n_jobs = -1, max_depth=12)
rf.fit(X_train,y_train.ravel([1]))

predicted_Y = rf.predict(X_test)


In [488]:
#pca = decomposition.RandomizedPCA(n_components=3)
#pca.fit(X_train)
#X_train_pca = pca.transform(X_train)
#x_test_pca = pca.transform(X_test)

#svc = svm.SVC(kernel='rbf', C=1e9, gamma=1e3)
#svc.fit(X_train,y_train.ravel([1]))
#svc.fit(X_train_pca,y_train.ravel([1]))

#predicted_svc = svc.predict(x_test_pca)

In [489]:
#useful
print rf.score(X_train,y_train)
y_test_ravel = y_test.ravel([1])
#print y_test_ravel,'\n',predicted_Y

#print 'x'*25

#print svc.score(X_train_pca,y_train)
#y_test_ravel = y_test.ravel([1])
#print y_test_ravel,'\n',predicted_svc

print(metrics.f1_score(y_test_ravel, predicted_Y))
#print(metrics.f1_score(y_test_ravel, predicted_svc))


0.471615720524
0.269242096187


C:\Prajeen\_Workstation\Anaconda\lib\site-packages\sklearn\metrics\classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


In [490]:

print metrics.confusion_matrix(predicted_Y,y_test_ravel)
#print metrics.confusion_matrix(predicted_kn,y_test_ravel)

[[20  8  7  4  2  0]
 [ 1  4  7  4  2  0]
 [ 1  2  0  0  0  0]
 [ 0  2  1  2  2  0]
 [ 0  3  1  4  2  3]
 [ 0  5  4  3  8 11]]


In [491]:
print metrics.classification_report(predicted_Y,y_test_ravel)
#print metrics.classification_report(predicted_kn,y_test_ravel)

             precision    recall  f1-score   support

        0.0       0.91      0.49      0.63        41
        1.0       0.17      0.22      0.19        18
        2.0       0.00      0.00      0.00         3
        3.0       0.12      0.29      0.17         7
        4.0       0.12      0.15      0.14        13
        5.0       0.79      0.35      0.49        31

avg / total       0.59      0.35      0.42       113



In [218]:
#print to the file
fileName = 'algoResult.txt'
#initialRead =  open(fileName)
#print initialRead.read()

file =  open(fileName, 'w')
file.truncate()

for i in range(predicted_Y.size):
    file.write(repr(int(predicted_Y[i]))+','+'\n')
    
file.close()

In [929]:
#Notes
# do confusion_matrix and classification_report from 4.3 -- done
# implement train_test_split from 4.3 , check also 6.1 -- done
# implement 4.3 Exercise: Model Selection via Validation -- done
# implement 5.1 PCA and SVM (also from 7.1) - done

# implement GridSearchCV from 6.2 and 7.2
